In [1]:
# A new CSV file, named state_esti.csv, was created using the transaction data file.
import pandas as pd
import numpy as np
tran_data=pd.read_csv("transaction_data.csv")
tran_data_new = tran_data[["household_key","DAY","COUPON_MATCH_DISC"]]
tran_data_new_1 = tran_data[["household_key","DAY","SALES_VALUE"]]
tran_data_new_2 = tran_data[["household_key","DAY","QUANTITY"]]
newdata_1=tran_data_new_1.to_numpy()
newdata=tran_data_new.to_numpy()
newdata_2=tran_data_new_2.to_numpy()


In [2]:
dct = {}
for i in range(len(newdata_1)):
    temp_tup = (newdata_1[i][0], newdata_1[i][1])

    if temp_tup not in dct:
        dct[temp_tup] = [newdata_1[i][2]]
    else:
        dct[temp_tup].append(newdata_1[i][2])

ans = dict(sorted(dct.items()))


In [3]:
total_sell=[]
for key,value in ans.items():
    total_sell.append(int(sum(value)))

In [4]:
customer_purchase_day={}
for k in range(len(newdata)):
    tple=(newdata[k][0],newdata[k][1])
    if tple not in customer_purchase_day:
        customer_purchase_day[tple]=[newdata[k][2]]
    else:
        customer_purchase_day[tple].append(newdata[k][2])
customer_purchase_day=dict(sorted(customer_purchase_day.items()))

In [5]:
is_coupon_redem=[]
number_of_coupon_redem=[]
coupon_redem=[]
for key,value in customer_purchase_day.items():
    non_zero_elements = [x for x in value if x != 0]
    number_of_coupon_redem.append(len(non_zero_elements))
    if sum(value)!=0:
        is_coupon_redem.append(1)
    else:
        is_coupon_redem.append(0)
    coupon_redem.append(-1*(sum(value)))
        
    

In [6]:
sandick = {}
for i in range(len(newdata_2)):
    temp_tup = (newdata_2[i][0], newdata_2[i][1])

    if temp_tup not in sandick:
        sandick[temp_tup] = [newdata_2[i][2]]
    else:
        sandick[temp_tup].append(newdata_2[i][2])

ans = dict(sorted(sandick.items()))


In [7]:
number_of_product_sell=[]
for key_2,value_2 in ans.items():
    number_of_product_sell.append(sum(value_2))

In [8]:
household_key=[]
tran_day=[]
for key,value in customer_purchase_day.items():
    tran_day.append(int(key[1]))
    household_key.append(int(key[0]))

In [9]:
data={'household_key':household_key,'tran_day':tran_day,'is_coupon_redem':is_coupon_redem}
dataset = pd.DataFrame(data)
dataset["prev_tran_day"]=dataset.groupby('household_key')["tran_day"].shift(periods=1,fill_value=0)
dataset["days_since_last_transaction"]=np.where(dataset["prev_tran_day"]!=0,dataset["tran_day"]-dataset["prev_tran_day"],0)
dataset['average_number_of_days_between_transaction'] = dataset.groupby('household_key')['days_since_last_transaction'].rolling(window=200, min_periods=1).mean().reset_index(0, drop=True)
dataset["future_tran_day"]=dataset.groupby('household_key')["tran_day"].shift(periods=-1,fill_value=0)
dataset["number_of_coupon_redem"]=number_of_coupon_redem


In [10]:
dataset["average_number_of_days_between_transaction"]=dataset["average_number_of_days_between_transaction"].apply(np.int64)
dataset["Coupan_redemtion_percentage_in_last_transaction"]=dataset.groupby("household_key")["is_coupon_redem"].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
dataset=dataset.round({"Coupan_redemtion_percentage_in_last_transaction":2})
dataset["tran_days_to_future_tran_day"]=np.where(dataset["future_tran_day"]!=0,dataset["future_tran_day"]-dataset["tran_day"],0)
#dataset["purchase_with_in_one_week"]=np.where(dataset["tran_days_to_future_tran_day"]<=7,1,0)
dataset["number_of_product"]=number_of_product_sell
dataset["average_number_of_product"]=dataset.groupby("household_key")["number_of_product"].rolling(window=200, min_periods=1).mean().reset_index(0, drop=True)
dataset["average_number_of_product"]=dataset["average_number_of_product"].apply(np.int64)
dataset["tran_value"]=total_sell
dataset["coupon_redem"]=coupon_redem
dataset["CDP"]=(dataset["coupon_redem"]/dataset["tran_value"])*100
dataset = dataset.replace([np.inf, -np.inf,np.nan], 0)
dataset['CDP'] = np.ceil(dataset['CDP'])
# define the bins and labels
bins = [-1,0,5,10,15,50]
labels = [0,1,5,10,15]

# replace the values in the column based on the given condition
dataset['CDP'] = pd.cut(dataset['CDP'], bins=bins, labels=labels)

dataset["average_tran_value"]=dataset.groupby("household_key")["tran_value"].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
dataset["average_tran_value"]=dataset["average_tran_value"].apply(np.int64)
#dataset["next_tran_value"]=dataset.groupby('household_key')["tran_value"].shift(periods=-1,fill_value=0)
#dataset['match'] = dataset.household_key.eq(dataset.household_key.shift(-1))
#dataset = dataset[dataset['match'] == True]
dataset.to_csv("state_esti.csv",index=False)


In [11]:
# demographic_data=pd.read_csv("hh_demographic.csv")
# dataset = pd.merge(dataset,demographic_data,on="household_key",how="left")
# encoded = pd.get_dummies(dataset[dataset.columns[dataset.dtypes==object]])
# data_encoded = pd.concat([encoded, dataset[dataset.columns[dataset.dtypes != object]]],axis=1)
# data_encoded.to_csv("data_processing.csv",index=False)